# Построение модели на примере полиномиальной регрессии (polynomial regression) с использованием `sklearn.pipeline`

## Подготовка окружения

In [1]:
# ВНИМАНИЕ: необходимо удостовериться, что виртуальная среда выбрана правильно!

# Для MacOS/Ubuntu
# !which pip

# Для Windows
# !where pip

In [2]:
# !conda install matplotlib numpy scikit-learn seaborn scipy -y

In [3]:
import numpy as np

np.__version__

'1.19.2'

In [4]:
import pandas as pd

pd.__version__

'1.2.3'

In [5]:
import scipy
from scipy import stats

scipy.__version__

'1.6.2'

In [6]:
import matplotlib
import matplotlib.pyplot as plt

matplotlib.__version__

'3.3.4'

In [7]:
import seaborn as sns

sns.__version__

'0.11.1'

## Загрузка данных

[Источник (FuelConsumption)](https://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64)

In [8]:
df = pd.read_csv("./../../data/FuelConsumptionCo2.csv")
df

,MODELYEAR,MAKE,MODEL,VEHICLECLASS,ENGINESIZE,CYLINDERS,TRANSMISSION,FUELTYPE,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
0,2014,ACURA,ILX,COMPACT,2.0,4,AS5,Z,9.9,6.7,8.5,33,196
1,2014,ACURA,ILX,COMPACT,2.4,4,M6,Z,11.2,7.7,9.6,29,221
2,2014,ACURA,ILX HYBRID,COMPACT,1.5,4,AV7,Z,6.0,5.8,5.9,48,136
3,2014,ACURA,MDX 4WD,SUV - SMALL,3.5,6,AS6,Z,12.7,9.1,11.1,25,255
4,2014,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS6,Z,12.1,8.7,10.6,27,244
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062,2014,VOLVO,XC60 AWD,SUV - SMALL,3.0,6,AS6,X,13.4,9.8,11.8,24,271
1063,2014,VOLVO,XC60 AWD,SUV - SMALL,3.2,6,AS6,X,13.2,9.5,11.5,25,264
1064,2014,VOLVO,XC70 AWD,SUV - SMALL,3.0,6,AS6,X,13.4,9.8,11.8,24,271
1065,2014,VOLVO,XC70 AWD,SUV - SMALL,3.2,6,AS6,X,12.9,9.3,11.3,25,260


## Pipeline

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

In [10]:
y = df['CO2EMISSIONS'].copy()
X = df.loc[:, df.columns != 'CO2EMISSIONS'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
# TODO: Outliers

In [12]:
numeric_transformer = Pipeline(steps=[
    ('scaler', RobustScaler())
])

In [13]:
CYLINDERS_transformer = Pipeline(steps=[
    ('replace', FunctionTransformer(lambda x: x.replace(to_replace=
                                                           {3: 4, 
                                                            5: 4, 
                                                            10: 8,
                                                            12: 8}))),
     ('encoder_ord', OrdinalEncoder())])
CYLINDERS_transformer

Pipeline(steps=[('replace',
                 FunctionTransformer(func=<function <lambda> at 0x7ff76e22f3a0>)),
                ('encoder_ord', OrdinalEncoder())])

In [14]:
ENGINESIZE_transformer = Pipeline(steps=[
    ('round', FunctionTransformer(lambda x: x.astype(float).round(0))),
    ('replace', FunctionTransformer(lambda x: x.replace(to_replace={1: 2, 
                                                                    7: 8, 
                                                                    8: 6}))),
    ('encoder', OrdinalEncoder())
    ])
ENGINESIZE_transformer

Pipeline(steps=[('round',
                 FunctionTransformer(func=<function <lambda> at 0x7ff76a1e09d0>)),
                ('replace',
                 FunctionTransformer(func=<function <lambda> at 0x7ff76a1e0af0>)),
                ('encoder', OrdinalEncoder())])

In [15]:
FUELTYPE_transformer = Pipeline(steps=[
    ('replace', FunctionTransformer(lambda x: x.replace('D', 'X'))), 
    ('encoder_oh', OneHotEncoder()) 
    ])
FUELTYPE_transformer

Pipeline(steps=[('replace',
                 FunctionTransformer(func=<function <lambda> at 0x7ff76a1e0dc0>)),
                ('encoder_oh', OneHotEncoder())])

In [16]:
preprocessor = ColumnTransformer(
    transformers=[
        ('drop', 'drop', df.columns.difference(['FUELCONSUMPTION_COMB_MPG', 'CYLINDERS', 'ENGINESIZE', 'FUELTYPE', 'CO2EMISSIONS']).tolist()),
        ('categorical_CYLINDERS', CYLINDERS_transformer, ['CYLINDERS']),
        ('categorical_ENGINESIZE', ENGINESIZE_transformer, ['ENGINESIZE']),
        ('categorical_FUELTYPE', FUELTYPE_transformer, ['FUELTYPE']),
        ('numeric_FUELCONSUMPTION_COMB_MPG', numeric_transformer, ['FUELCONSUMPTION_COMB_MPG']),         
]) 
preprocessor

ColumnTransformer(transformers=[('drop', 'drop',
                                 ['FUELCONSUMPTION_CITY',
                                  'FUELCONSUMPTION_COMB', 'FUELCONSUMPTION_HWY',
                                  'MAKE', 'MODEL', 'MODELYEAR', 'TRANSMISSION',
                                  'VEHICLECLASS']),
                                ('categorical_CYLINDERS',
                                 Pipeline(steps=[('replace',
                                                  FunctionTransformer(func=<function <lambda> at 0x7ff76e22f3a0>)),
                                                 ('encoder_ord',
                                                  OrdinalEncoder())]),
                                 ['CYLINDERS']),
                                ('categoric...
                                                  FunctionTransformer(func=<function <lambda> at 0x7ff76a1e0af0>)),
                                                 ('encoder',
                                    

In [17]:
pipeline = Pipeline(steps = [
    ('preprocessor', preprocessor),
    ('poly', PolynomialFeatures(degree=2)),
    ('linear_regressor', LinearRegression())
])
model = pipeline.fit(X_train, y_train)
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('drop', 'drop',
                                                  ['FUELCONSUMPTION_CITY',
                                                   'FUELCONSUMPTION_COMB',
                                                   'FUELCONSUMPTION_HWY',
                                                   'MAKE', 'MODEL', 'MODELYEAR',
                                                   'TRANSMISSION',
                                                   'VEHICLECLASS']),
                                                 ('categorical_CYLINDERS',
                                                  Pipeline(steps=[('replace',
                                                                   FunctionTransformer(func=<function <lambda> at 0x7ff76e22f3a0>)),
                                                                  ('encoder_ord',
                                                                   OrdinalEncoder...
       

In [18]:
from sklearn.metrics import r2_score

# оценить модель
y_predicted = model.predict(X_test)

print("Mean absolute error: %.2f" % np.mean(np.absolute(y_predicted - y_test)))
print("Residual sum of squares (MSE): %.2f" % np.mean((y_predicted - y_test) ** 2))
print("R2-score: %.2f" % r2_score(y_true=y_test, y_pred=y_predicted))

Mean absolute error: 5.14
Residual sum of squares (MSE): 72.53
R2-score: 0.98
